<a href="https://colab.research.google.com/github/Brian0718UOA/2021-Summer-Research-Scholarship/blob/main/bk_yolov5_NZTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# change directory to yolov5 folder
%cd drive/MyDrive/Colab \Notebooks/yolov5

/content/drive/MyDrive/Colab Notebooks/yolov5


In [ ]:
# do not need to clone (i.e run this cell) if yolov5 folder is uploaded to Google Colab
# !git clone https://github.com/ultralytics/yolov5.git
# %cd yolov5/
# !pip install -U -r requirements.txt

# Upload files as zip files from local machine

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import shutil
shutil.move("/content/NZTA.zip", "/content/yolov5/data")  # move NZTA.zip file into data folder

In [ ]:
%cd yolov5/data/
!unzip NZTA.zip  # zip file to folder with images

# Upload files from URL

In [ ]:
# import requests

# url = input("paste jpg url: ")

# response = requests.get(url, stream = True)

# text_file = open("NZTA_cctv.jpg", "wb")
# for chunk in response.iter_content(chunk_size = 1024):
#   text_file.write(chunk)


# text_file.close()

paste jpg url: https://www.trafficnz.info/camera/653.jpg


In [ ]:
# # for saving multiple urls as jpg into destined folder so that they can be inferenced together
# import requests

# # save a list of urls into a list and convert the urls to jpg
# url_list = [] 
# url = input("input jpg url: ")
# while url != "done":
#   url_list.append(url)
#   url = input("input jpg url or type done if no more inputs: ")
# print(url_list)

# for url in url_list:
#   response = requests.get(url, stream = True)

#   jpg_file = open('/content/sample_data/'+str(url[-7:-4])+'.jpg', 'wb')  # change path to desired  
#   for chunk in response.iter_content(chunk_size = 1024):
#     jpg_file.write(chunk)
#   jpg_file.close()


In [ ]:
# reading url from csv file
import pandas as pd
import requests

url_df = pd.read_csv("Traffic-state-cctv/Brian SRS - NZTA traffic camera.csv", header=None)
url_df.columns = ['location','url']
url_df

for url in url_df.url:
  response = requests.get(url, stream = True)

  jpg_file = open('/content/sample_data/'+str(url[-7:-4])+'.jpg', 'wb')  # change path to desired  
  for chunk in response.iter_content(chunk_size = 1024):
    jpg_file.write(chunk)
  jpg_file.close()  
  

# Inference

In [ ]:
# inference using COCO pretrained weights
!python detect.py --weights yolov5s.pt --img 640 --conf 0.6 --source /content/sample_data/  # change source to directory with images to inference

In [ ]:
# inferencing the image from url
!python detect.py --weights yolov5s.pt --img 640 --conf 0.2 --source NZTA_cctv.jpg

## Training

In [ ]:
# run in yolov5 directory
%pip install -q roboflow

import torch  # go to Runtime then change runtime type and change hardware accelerator to GPU
import os
# from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.10.0+cu111 (Tesla K80)


In [ ]:
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics


In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [ ]:
# after following the link above, recieve python code with these fields filled in
# custom dataset uploaded to personal roboflow account
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="N4HxG7yvCWRQU1JYCNKC")
project = rf.workspace().project("vehicles-openimages-xrwcz")
dataset = project.version(1).download("yolov5")

In [ ]:
# train our yolov5 model with custom dataset
# we choose our weights to start transfer learning from the COCO pretrained checkpoint yolov5s.pt
!python train.py --img 640 --batch 32 --epochs 300 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache

In [ ]:
# export your model's weights for future use
from google.colab import files
files.download('./runs/train/exp4/weights/best.pt')  # saved to local machine

In [ ]:
# Inference with trained model on test images
!python detect.py --weights runs/train/exp4/weights/best.pt --img 416 --conf 0.1 --source {dataset.location}/test/images

In [ ]:
# retry inferencing the images from nzta cctv
# uploaded trained weights best.pt to yolov5 folder on gdrive
!python detect.py --weights best.pt --img 640 --conf 0.6 --source /content/sample_data/  

## Data Augmentation

In [ ]:
# import os

# # create new folder to save augmented images
# newfolder = r'/content/drive/MyDrive/Colab Notebooks/yolov5/aug_images'
# os.makedirs(newfolder)


In [ ]:
# from google.colab import files
# uploaded = files.upload()

Saving bus_img_trial.zip to bus_img_trial.zip


In [ ]:
# import shutil
# shutil.move('bus_img_trial.zip', "/aug_images")  # copy and paste file dir

# %cd aug_images/
# !unzip bus_img_trial.zip  # copy and paste file name containing dataset

/content/drive/MyDrive/Colab Notebooks/yolov5/aug_images
Archive:  bus_img_trial.zip
  inflating: 1612315654305.jpg       
  inflating: ebus-exterior-sustainability-compressed.jpg  
  inflating: tamakilink-bus-around-auckland-v1.jpg  


In [28]:
import cv2
import os
import numpy as np
from skimage.util import random_noise
from skimage.transform import rotate
from scipy import ndimage
from skimage.filters import gaussian
from skimage import img_as_ubyte
from PIL import Image, ImageEnhance

counter = 0
directory = "/content/drive/MyDrive/Colab Notebooks/yolov5/img_to_augment/"
os.chdir(directory)  # change directory to where augmented images are stored
for filename in os.listdir(directory):
  if filename.endswith(".jpg"):
    img = cv2.imread(directory+filename)
    
    # flipped_image = cv2.flip(img, 1)  # flip image horizontally
    # cv2.imwrite('flipped'+str(counter)+".jpg", flipped_image)
    
    # noise_image = random_noise(img, mode='s&p', amount=0.2)  # add salt and pepper noise to image
    # noise_image = np.array(255*noise_image, dtype = 'uint8')
    # cv2.imwrite('noise'+str(counter)+".jpg", noise_image)

    # rotate_image_cw = rotate(img, angle=-15)  # rotate image by 15 degrees cw
    # rotate_image_cw = np.array(255*rotate_image_cw, dtype = 'uint8')  
    # rotate_image_ccw = rotate(img, angle=15)  # rotate image by 15 degrees ccw
    # rotate_image_ccw = np.array(255*rotate_image_ccw, dtype = 'uint8')
    # cv2.imwrite('rotate_cw'+str(counter)+".jpg", rotate_image_cw)
    # cv2.imwrite('rotate_ccw'+str(counter)+".jpg", rotate_image_ccw)

    # blur_image = ndimage.uniform_filter(img, size=(11, 11, 1))  # blur image
    # cv2.imwrite('blur'+str(counter)+".jpg", blur_image)

    # dark_image = img * (100/255)  # darken image
    # cv2.imwrite('darker'+str(counter)+'.jpg', dark_image)

    enhancer = ImageEnhance.Brightness(img)
    bright_image = enhancer.enhance(1.5)
    cv2.imwrite('brighter'+str(counter)+'.jpg', bright_image)
    dark_image = enhancer.enhance(0.5)
    cv2.imwrite('darker'+str(counter)+'.jpg', dark_image)

    counter += 1
  